In [159]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os

def clean_host(host):
    return host.split('.')[-2]

def get_site(site_url):
    return site_url.split(".")[1]

def is_third_party(origin, cookie_host):
    if origin in cookie_host:
        return False
    return True


In [174]:
sql_file = 'exp-data/nyt-c1.sqlite' 
    
###### javascript cookies  ######
query = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """

conn = sqlite3.connect(sql_file)

cookies = pd.read_sql_query(query, conn)
cookies

,site_url,visit_id,host,name,value
0,https://www.webmd.com/,1,.www.webmd.com,__cfduid,dd673b2700baa134208e3794c02eec1f41606506583
1,https://www.webmd.com/,1,.webmd.com,lrt_wrk,lrt1_cached_k8_worker_1_37S_21O_2020-11-18_17:...
2,https://www.webmd.com/,1,.webmd.com,gtinfo,"{""ct"":""Columbus"",""c"":""Franklin"",""cc"":""39049"",""..."
3,https://www.webmd.com/,1,.webmd.com,VisitorId,9831e05f-086b-4123-5987-f9b5df875967
4,https://www.webmd.com/,1,.css.webmd.com,__cfduid,dc560e081e9789ea111079dc63ad819771606506583
...,...,...,...,...,...
1391,https://www.nytimes.com/,3,.yahoo.com,GUC,AQEBAQFfwqtfy0IcvQQZ
1392,https://www.nytimes.com/,3,.pubmatic.com,KTPCACOOKIE,YES
1393,https://www.nytimes.com/,3,.analytics.yahoo.com,IDSYNC,18y3~1v0j
1394,https://www.nytimes.com/,3,.ads.pubmatic.com,KCCH,YES


In [175]:
cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies["host"] = list(map(lambda x: clean_host(x), cookies.host.tolist()))
cookies["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

In [176]:
# 95 unique host -> after clean the host: 67
# cookies.host.unique()

In [177]:
cookies = cookies[["visit_id", "origin_site", "host", "name", "value", "is_third_party"]]

In [178]:
# drop duplicate: 1396 -> 539
cookies = cookies.drop_duplicates().reset_index(drop = True)

In [179]:
cookies

,visit_id,origin_site,host,name,value,is_third_party
0,1,webmd,webmd,__cfduid,dd673b2700baa134208e3794c02eec1f41606506583,False
1,1,webmd,webmd,lrt_wrk,lrt1_cached_k8_worker_1_37S_21O_2020-11-18_17:...,False
2,1,webmd,webmd,gtinfo,"{""ct"":""Columbus"",""c"":""Franklin"",""cc"":""39049"",""...",False
3,1,webmd,webmd,VisitorId,9831e05f-086b-4123-5987-f9b5df875967,False
4,1,webmd,webmd,__cfduid,dc560e081e9789ea111079dc63ad819771606506583,False
...,...,...,...,...,...,...
534,3,nytimes,yahoo,A3,d=AQABBIdZwV8CEAePsjTU5cRNE7tmBgFkr9IFEgEBAQGr...,True
535,3,nytimes,pubmatic,KTPCACOOKIE,YES,True
536,3,nytimes,yahoo,IDSYNC,18y3~1v0j,True
537,3,nytimes,pubmatic,KCCH,YES,True


In [180]:
# cookies[cookies.is_third_party & (cookies.visit_id == 3)]
cookies[(cookies.visit_id == 3)].shape[0]

100

In [144]:
cookies.groupby('host',as_index=False)["host", "name", "value"].size()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


host
1rx                 4
3lift               8
acuityplatform      3
addthis             4
adentifi            4
                 ... 
webmd             154
yahoo              21
yieldmo             7
youtube             2
zemanta             2
Length: 99, dtype: int64

In [142]:
# make sure to use brackets when filtering visit_id == 3
temp = cookies[(cookies.visit_id == 3) & cookies.is_third_party]
temp.groupby('host',as_index=False)["host", "name", "value"].size().nlargest(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


host
pubmatic          18
rubiconproject    14
media              8
adsrvr             7
casalemedia        5
dtype: int64

In [131]:
cookies.groupby('origin_site')["host", "name", "value"].nunique()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,host,name,value
origin_site,,,
mayoclinic,11,47,76
nytimes,25,80,94
webmd,57,186,322


In [20]:
# cookies.groupby('origin_site').size()

In [16]:
# big_table = []

# data_path = "exp-data/"

# def extract_unique_cookie_pairs(type_name, exp_group, exp_num):
#     file_name = type_name + '-' + exp_group + str(exp_num)
#     print(file_name)
#     conn = sqlite3.connect(data_path + file_name +".sqlite")
#     javascript_cookies = pd.read_sql_query(
#     "select visit_id, host, name, value from javascript_cookies;", conn)
#     javascript_cookies = javascript_cookies.drop_duplicates()
#     df = javascript_cookies.groupby('visit_id')["host", "name", "value"].nunique()
#     pairs = javascript_cookies.groupby('visit_id').size().values
#     df["pairs"] = pairs
#     df['origin_site'] = ['webmd', 'mayoclinic', type_name]
#     df['exp_type'] = [type_name] * 3
#     df['exp_group'] = [exp_group] * 3
#     df['exp_index'] = [exp_num] * 3
#     return df

In [19]:
# exp_types = ['nyt', 'forbes', 'washington']
# exp_groups = ['t', 'c']
# for i in range(1, 7):
#     for t in exp_types:
#         for g in exp_groups:
#             big_table.append(extract_unique_cookie_pairs(t, g, i))

In [18]:
# result = pd.concat(big_table)

In [21]:
# result.to_csv("cookie_result-6.csv", index=False)

In [181]:
# result